# ModelChain

## A simple ModelChain example

Before delving into the intricacies of ModelChain, we provide a brief example of the modeling steps using ModelChain. First, we import pvlib's objects, module data, and inverter data.

In [1]:
import pandas as pd
import numpy as np

# pvlib imports
import pvlib

from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain

# load some module and inverter specifications
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']

Now we create a Location object, a PVSystem object, and a ModelChain object.

In [2]:
location = Location(latitude=32.2, longitude=-110.9)
system = PVSystem(surface_tilt=20, surface_azimuth=200, 
                  module_parameters=sandia_module,
                  inverter_parameters=cec_inverter)
mc = ModelChain(system, location)

Printing a ModelChain object will display its models.

In [3]:
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


Next, we run a model with some simple weather data.

In [4]:
weather = pd.DataFrame([[1050, 1000, 100, 30, 5]], 
                       columns=['ghi', 'dni', 'dhi', 'temp_air', 'wind_speed'], 
                       index=[pd.Timestamp('20170401 1200', tz='US/Arizona')])

mc.run_model(times=weather.index, weather=weather);

ModelChain stores the modeling results on a series of attributes. A few examples are shown below.

In [5]:
mc.aoi

2017-04-01 12:00:00-07:00    15.929176
Name: aoi, dtype: float64

In [6]:
mc.dc

,i_sc,i_mp,v_oc,v_mp,p_mp,i_x,i_xx
2017-04-01 12:00:00-07:00,5.485958,4.860317,52.319047,40.585752,197.259628,5.363079,3.401315


In [7]:
mc.ac

2017-04-01 12:00:00-07:00    189.915445
dtype: float64

The remainder of this guide examines the ModelChain functionality and explores common pitfalls.

## Defining a ModelChain

Let's make the most basic Location and PVSystem objects and build from there.

In [8]:
location = Location(32.2, -110.9)
poorly_specified_system = PVSystem()
print(location)
print(poorly_specified_system)

Location: 
  name: None
  latitude: 32.2
  longitude: -110.9
  altitude: 0
  tz: UTC
PVSystem: 
  name: None
  surface_tilt: 0
  surface_azimuth: 180
  module: None
  inverter: None
  albedo: 0.25
  racking_model: open_rack_cell_glassback


These basic objects do not have enough information for ModelChain to be able to automatically determine its set of models, so the ModelChain will throw an error when we try to create it.

In [9]:
ModelChain(poorly_specified_system, location)

ValueError: could not infer DC model from system.module_parameters

If our goal is simply to get the object constructed, we can specify the models that the ModelChain should use. We'll have to fill in missing data on the PVSystem object later, but maybe that's desirable in some workflows.

In [10]:
mc = ModelChain(poorly_specified_system, location, 
                dc_model='singlediode', ac_model='snlinverter', 
                aoi_model='physical', spectral_model='no_loss')
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: singlediode
  ac_model: snlinverter
  aoi_model: physical_aoi_loss
  spectral_model: no_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


In [11]:
mc.run_model(times=weather.index, weather=weather)

TypeError: calcparams_desoto() missing 6 required positional arguments: 'alpha_sc', 'a_ref', 'I_L_ref', 'I_o_ref', 'R_sh_ref', and 'R_s'

Next, we define a PVSystem with a module from the SAPM database and an inverter from the CEC database. ModelChain will examine the PVSystem object's properties and determine that it should choose the SAPM DC model, AC model, AOI loss model, and spectral loss model.

In [12]:
sapm_system = PVSystem(module_parameters=sandia_module, inverter_parameters=cec_inverter)
mc = ModelChain(system, location)
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


In [13]:
mc.run_model(times=weather.index, weather=weather)
mc.ac

2017-04-01 12:00:00-07:00    189.915445
dtype: float64

Alternatively, we could have specified single diode or PVWatts related information in the PVSystem construction. Here we pass PVWatts data to the PVSystem. ModelChain will automatically determine that it should choose PVWatts DC and AC models. ModelChain still needs us to specify ``aoi_model`` and ``spectral_model`` keyword arguments because the ``system.module_parameters`` dictionary does not contain enough information to determine which of those models to choose.

In [14]:
pvwatts_system = PVSystem(module_parameters={'pdc0': 240, 'gamma_pdc': -0.004})
mc = ModelChain(pvwatts_system, location, 
                aoi_model='physical', spectral_model='no_loss')
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: pvwatts_dc
  ac_model: pvwatts_inverter
  aoi_model: physical_aoi_loss
  spectral_model: no_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


In [15]:
mc.run_model(times=weather.index, weather=weather)
mc.ac

2017-04-01 12:00:00-07:00    198.519999
dtype: float64

User-supplied keyword arguments override ModelChain's inspection methods. For example, we can tell ModelChain to use different loss functions for a PVSystem that contains SAPM-specific parameters. 

In [16]:
sapm_system = PVSystem(module_parameters=sandia_module, inverter_parameters=cec_inverter)
mc = ModelChain(system, location, aoi_model='physical', spectral_model='no_loss')
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: physical_aoi_loss
  spectral_model: no_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


In [17]:
mc.run_model(times=weather.index, weather=weather)
mc.ac

2017-04-01 12:00:00-07:00    191.991429
dtype: float64

## Demystifying ModelChain internals

### run_model

In [18]:
np.source(mc.run_model)

In file: /home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/modelchain.py

    def run_model(self, times=None, weather=None):
        """
        Run the model.

        Parameters
        ----------
        times : None or DatetimeIndex, default None
            Times at which to evaluate the model. Can be None if attribute
            `times` is already set.
        weather : None or DataFrame, default None
            If None, assumes air temperature is 20 C, wind speed is 0 m/s and
            irradiation calculated from clear sky data.
            For conventional weather data, column names must be 'wind_speed',
            'temp_air', 'dni', 'ghi', 'dhi'. Do not pass incomplete irradiation
            data, instead use method
            :py:meth:`~pvlib.modelchain.ModelChain.complete_irradiance`.
            For complete PV reference device data, column names must be 'F' and
            'H', for effective irradiance ratio and effective temperature
            ratio, resp

### Wrapping methods into a unified API

Readers may notice that the source code of the ModelChain.run_model method is model-agnostic. ModelChain.run_model calls generic methods such as ``self.dc_model`` rather than a specific model such as ``singlediode``. So how does the ModelChain.run_model know what models it's supposed to run? The answer comes in two parts, and allows us to explore more of the ModelChain API along the way.

First, ModelChain has a set of methods that wrap the PVSystem methods that perform the calculations (or further wrap the pvsystem.py module's functions). Each of these methods takes the same arguments (``self``) and sets the same attributes, thus creating a uniform API. For example, the ModelChain.pvwatts_dc method is shown below. Its only argument is ``self``, and it sets the ``dc`` attribute.

In [19]:
np.source(mc.pvwatts_dc)

In file: /home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/modelchain.py

    def pvwatts_dc(self):
        self.dc = self.system.pvwatts_dc(self.effective_irradiance,
                                         self.temps['temp_cell'])
        return self



The ModelChain.pvwatts_dc method calls the pvwatts_dc method of the PVSystem object that we supplied using data that is stored in its own ``effective_irradiance`` and ``temps`` attributes. Then it assigns the result to the ``dc`` attribute of the ModelChain object. The code below shows a simple example of this.

In [20]:
# make the objects
pvwatts_system = PVSystem(module_parameters={'pdc0': 240, 'gamma_pdc': -0.004})
mc = ModelChain(pvwatts_system, location, 
                aoi_model='no_loss', spectral_model='no_loss')

# manually assign data to the attributes that ModelChain.pvwatts_dc will need.
# for standard workflows, run_model would assign these attributes.
mc.effective_irradiance = pd.Series(1000, index=[pd.Timestamp('20170401 1200-0700')])
mc.temps = pd.DataFrame({'temp_cell': 50, 'temp_module': 50}, index=[pd.Timestamp('20170401 1200-0700')])

# run ModelChain.pvwatts_dc and look at the result
mc.pvwatts_dc()
mc.dc

2017-04-01 12:00:00-07:00    216.0
dtype: float64

The ModelChain.sapm method works similarly to the ModelChain.pvwatts_dc method. It calls the PVSystem.sapm method using stored data, then assigns the result to the ``dc`` attribute. The ModelChain.sapm method differs from the ModelChain.pvwatts_dc method in three notable ways. First, the PVSystem.sapm method expects different units for effective irradiance, so ModelChain handles the conversion for us. Second, the PVSystem.sapm method (and the PVSystem.singlediode method) returns a DataFrame with current, voltage, and power parameters rather than a simple Series of power. Finally, this current and voltage information allows the SAPM and single diode model paths to support the concept of modules in series and parallel, which is handled by the PVSystem.scale_voltage_current_power method. 

In [21]:
np.source(mc.sapm)

In file: /home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/modelchain.py

    def sapm(self):
        self.dc = self.system.sapm(self.effective_irradiance/1000.,
                                   self.temps['temp_cell'])

        self.dc = self.system.scale_voltage_current_power(self.dc)

        return self



In [22]:
# make the objects
sapm_system = PVSystem(module_parameters=sandia_module, inverter_parameters=cec_inverter)
mc = ModelChain(sapm_system, location)

# manually assign data to the attributes that ModelChain.sapm will need.
# for standard workflows, run_model would assign these attributes.
mc.effective_irradiance = pd.Series(1000, index=[pd.Timestamp('20170401 1200-0700')])
mc.temps = pd.DataFrame({'temp_cell': 50, 'temp_module': 50}, index=[pd.Timestamp('20170401 1200-0700')])

# run ModelChain.sapm and look at the result
mc.sapm()
mc.dc

,i_sc,i_mp,v_oc,v_mp,p_mp,i_x,i_xx
2017-04-01 12:00:00-07:00,5.14168,4.566863,53.8368,42.4284,193.764685,5.025377,3.219662


We've established that the ``ModelChain.pvwatts_dc`` and ``ModelChain.sapm`` have the same API: they take the same arugments (``self``) and they both set the ``dc`` attribute.\* Because the methods have the same API, we can call them in the same way. ModelChain includes a large number of methods that perform the same API-unification roles for each modeling step.

Again, so how does the ModelChain.run_model know which models it's supposed to run?

At object construction, ModelChain assigns the desired model's method (e.g. ``ModelChain.pvwatts_dc``) to the corresponding generic attribute (e.g. ``ModelChain.dc_model``) using a method described in the next section.

In [23]:
pvwatts_system = PVSystem(module_parameters={'pdc0': 240, 'gamma_pdc': -0.004})
mc = ModelChain(pvwatts_system, location, 
                aoi_model='no_loss', spectral_model='no_loss')
mc.dc_model.__func__

<function pvlib.modelchain.ModelChain.pvwatts_dc(self)>

The ModelChain.run_model method can ignorantly call ``self.dc_module`` because the API is the same for all methods that may be assigned to this attribute.

\* some readers may object that the API is *not* actually the same because the type of the ``dc`` attribute is different (Series vs. DataFrame)!

### Inferring models

How does ModelChain infer the appropriate model types? ModelChain uses a series of methods (ModelChain.infer_dc_model, ModelChain.infer_ac_model, etc.) that examine the user-supplied PVSystem object. The inference methods use set logic to assign one of the model-specific methods, such as ModelChain.sapm or ModelChain.snlinverter, to the universal method names ModelChain.dc_model and ModelChain.ac_model. A few examples are shown below.

In [24]:
np.source(mc.infer_dc_model)

In file: /home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/modelchain.py

    def infer_dc_model(self):
        params = set(self.system.module_parameters.keys())
        if set(['A0', 'A1', 'C7']) <= params:
            return self.sapm
        elif set(['a_ref', 'I_L_ref', 'I_o_ref', 'R_sh_ref', 'R_s']) <= params:
            return self.singlediode
        elif set(['pdc0', 'gamma_pdc']) <= params:
            return self.pvwatts_dc
        else:
            raise ValueError('could not infer DC model from '
                             'system.module_parameters')



In [25]:
np.source(mc.infer_ac_model)

In file: /home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/modelchain.py

    def infer_ac_model(self):
        inverter_params = set(self.system.inverter_parameters.keys())
        module_params = set(self.system.module_parameters.keys())
        if set(['C0', 'C1', 'C2']) <= inverter_params:
            return self.snlinverter
        elif set(['ADRCoefficients']) <= inverter_params:
            return self.adrinverter
        elif set(['pdc0']) <= module_params:
            return self.pvwatts_inverter
        else:
            raise ValueError('could not infer AC model from '
                             'system.inverter_parameters')



## User-defined models

Users may also write their own functions and pass them as arguments to ModelChain. The first argument of the function must be a ModelChain instance. For example, the functions below implement the PVUSA model and a wrapper function appropriate for use with ModelChain. This follows the pattern of implementing the core models using the simplest possible functions, and then implementing wrappers to make them easier to use in specific applications. Of course, you could implement it in a single function if you wanted to.

In [26]:
def pvusa(poa_global, wind_speed, temp_air, a, b, c, d):
    """
    Calculates system power according to the PVUSA equation
    
    P = I * (a + b*I + c*W + d*T)
    
    where
    P is the output power,
    I is the plane of array irradiance,
    W is the wind speed, and
    T is the temperature
    a, b, c, d are empirically derived parameters.
    """
    return poa_global * (a + b*poa_global + c*wind_speed + d*temp_air)


def pvusa_mc_wrapper(mc):
    # calculate the dc power and assign it to mc.dc
    mc.dc = pvusa(mc.total_irrad['poa_global'], mc.weather['wind_speed'], mc.weather['temp_air'],
                  mc.system.module_parameters['a'], mc.system.module_parameters['b'],
                  mc.system.module_parameters['c'], mc.system.module_parameters['d'])
    
    # returning mc is optional, but enables method chaining
    return mc


def pvusa_ac_mc_wrapper(mc):
    # keep it simple
    mc.ac = mc.dc
    return mc

In [27]:
module_parameters = {'a': 0.2, 'b': 0.00001, 'c': 0.001, 'd': -0.00005}
pvusa_system = PVSystem(module_parameters=module_parameters)

mc = ModelChain(pvusa_system, location, 
                dc_model=pvusa_mc_wrapper, ac_model=pvusa_ac_mc_wrapper,
                aoi_model='no_loss', spectral_model='no_loss')

A ModelChain object uses Python's functools.partial function to assign itself as the argument to the user-supplied functions.

In [28]:
mc.dc_model.func

<function __main__.pvusa_mc_wrapper(mc)>

The end result is that ModelChain.run_model works as expected!

In [29]:
mc.run_model(times=weather.index, weather=weather)
mc.dc

2017-04-01 12:00:00-07:00    209.519752
dtype: float64

## Campanelli et al. Model

The model of Campanelli et al. is available in the `pvsystem` module as a collection of lower-level functions that can be used in multiple ways with the `ModelChain` API.

### Use module parameters fit from data at pv-fit.com to define a PVSystem

The module parameters are typcially a direct mapping of the `fit_params` dict that is available from model calibrations performed on I-V curve calibration data `www.pv-fit.com`.

In [30]:
import pvlib.pvsystem as pvsystem

# dict defined from a JSON response subset of a pv-fit.com model calibration for a 72-cell, 275 W module at SRC
# TODO The 'fit_params' public API at pv-fit.com is not final. Finalized it before adding Campanelli model.
module_params_campanelli = {'i_sc_A_0': 8.300642232880616, 'i_rs_A_0': 3.1370554096867056e-10,
                            'n_mod_V_0': 1.8760873861375469, 'bg_mod_eV_0': 80.76475208957835,
                            'alpha_bg_mod_0': -0.0794653116276011, 'g_p_Mho_0': 0.0026062186459665454, 
                            'r_s_Ohm_0': 0.5297525695641349}

### Model wrapper for use with matched reference device effective irradiance and cell temperature

In this scenario, a reference device (typically a "matched" cell or module) provides the effecitive irradiance ratio `F` and effective temperature ratio `H`, which serves directly as the "weather" under which the device operates. The `F` and `H` are stored in the `ModelChain` object for further inspection and are cleared on subsequent model runs.

In [31]:
from pvlib.modelchain import sdm_campanelli

# Define the PVSystem and ModelChain
system_campanelli = PVSystem(surface_tilt=20, surface_azimuth=200,
                             module_parameters=module_params_campanelli,
                             inverter_parameters=cec_inverter)
weather_F_H = pd.DataFrame([[1.07754787, 1.084]], 
                           columns=['F', 'H',], 
                           index=[pd.Timestamp('20170401 1200', tz='US/Arizona')])
mc_campanelli = ModelChain(system_campanelli, location, dc_model=sdm_campanelli,
                           name="Campanelli with PV Reference Device",
                           aoi_model='no_loss', spectral_model='no_loss')
mc_campanelli.run_model(weather=weather_F_H)

ModelChain: 
  name: Campanelli with PV Reference Device
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: functools.partial(<function sdm_campanelli at 0x7f83471d07b8>, ModelChain: 
  name: Campanelli with PV Reference Device
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: ...
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses)
  ac_model: snlinverter
  aoi_model: no_aoi_loss
  spectral_model: no_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses

In [32]:
mc_campanelli.dc

,i_sc,v_oc,i_mp,v_mp,p_mp,i_x,i_xx
2017-04-01 12:00:00-07:00,8.944339,41.12082,8.246687,31.335752,258.416132,8.887189,5.505461


In [33]:
mc_campanelli.ac

2017-04-01 12:00:00-07:00    248.736184
dtype: float64

### Model wrapper for use with SAPM effective irradiance and cell temperature

When traditional weather data are available (e.g., from a meterological station), use the SAPM to compute the effecitive irradiance ratio `F` and effective temperature ratio `H`. Among other things, this relies on the
accuracy of the short-circuit temperature coefficient under the operating conditions of the given location. This computation uses the `Aisc` provided by the SAPM, and in practice, one tries to match as best as possible the module at hand with a module in the SAPM database.

Note that if either `F` or `H` is provided (but not both), then the SAPM will be used to compute the missing one.

In [34]:
module_params_campanelli_sapm = {**module_params_campanelli, **sandia_module.to_dict()}
module_params_campanelli_sapm['irrad_ref'] = 1000.  # W/m^2
module_params_campanelli_sapm['temp_ref'] = 25.  # degC
    
# Define the PVSystem and ModelChain
system_campanelli_sapm = PVSystem(surface_tilt=20, surface_azimuth=200,
                                  module_parameters=module_params_campanelli_sapm,
                                  inverter_parameters=cec_inverter)
mc_campanelli = ModelChain(system_campanelli_sapm, location, dc_model=sdm_campanelli,
                           name="Campanelli with SAPM")
# Run using the conventional weather data from above
mc_campanelli.run_model(times=weather.index, weather=weather)

ModelChain: 
  name: Campanelli with SAPM
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: functools.partial(<function sdm_campanelli at 0x7f83471d07b8>, ModelChain: 
  name: Campanelli with SAPM
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: ...
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses)
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses

In [35]:
mc_campanelli.F

array([1.07754787])

In [36]:
mc_campanelli.H

array([1.11097743])

In [37]:
# Because the SAPM cell temperature was warmer, the maximum power is decreased as compared to the previous result
mc_campanelli.dc

OrderedDict([('i_sc', array([8.94433932])),
             ('v_oc', array([39.82156774])),
             ('i_mp', array([8.20668637])),
             ('v_mp', array([30.06369951])),
             ('p_mp', array([246.72335314])),
             ('i_x', array([8.88651217])),
             ('i_xx', array([5.45186953]))])

In [38]:
mc_campanelli.ac

array([237.46198999])